In [ ]:
# This tutorial is available for download as a Jupyter notebook



# Source Data Simulation with SourceSimulator

This tutorial demonstrates how to use the :class:`~calibrain.SourceSimulator` class
to generate synthetic brain activity data for neuroimaging research (e.g., MEG/EEG 
source simulation). The SourceSimulator creates event-related potential (ERP)-like 
signals that can be used to test source localization algorithms, validate analysis 
pipelines, and benchmark uncertainty quantification methods.


In [ ]:
# Authors: Mohammad Orabe  <m.orabe@icloud.com>
# License: AGPL-3.0 license
# Copyright the CaliBrain contributors.

## Background

This tutorial focuses specifically on simulating stimulus-locked 
ERP-like signals at brain sources, which form the foundation for neuroimaging 
data simulation.

The SourceSimulator creates band-limited, windowed signals that mimic realistic 
event-related potentials (ERPs) time-locked to stimulus onset. The class can 
simulate multi-trial source time courses, where each trial contains a specific 
number of active sources at randomly selected locations. These simulated source
activities can then be used as input for forward modeling, inverse problem 
validation, and uncertainty quantification benchmarking.

**Scope:** The CaliBrain framework implements a comprehensive 7-stage pipeline for
neuroimaging simulation and validation. This tutorial covers **source-level 
simulation only** - the generation of neural activity patterns at cortical locations.
For projecting these sources to sensors using forward models and adding controlled 
noise, see the sensor simulation tutorial.

```text
┌───────────────┐    ┏━━━━━━━━━━━━━━━┓    ┌───────────────┐    ┌───────────────┐
│   Leadfield   │ →  ┃   ★ Source    ┃ →  │    Sensor     │ →  │    Source     │
│ Construction  │    ┃  Simulation   ┃    │  Simulation   │    │  Estimation   │
└───────────────┘    ┗━━━━━━━━━━━━━━━┛    └───────────────┘    └───────────────┘
                                                                      ↓
                     ┌───────────────┐    ┌───────────────┐    ┌───────────────┐
                     │  Evaluation   │ ←  │ Visualization │ ←  │  Uncertainty  │
                     │               │    │               │    │   Estimation  │
                     └───────────────┘    └───────────────┘    └───────────────┘
```


### Mathematical Formulation

Let $N$ denote the number of sources and $T$ the 
number of time points.

The ERP-like signal for each active source is generated through a 5-step process:

**1. Generate white noise:** Create a zero-mean white Gaussian noise sequence:

\begin{align}w(t) \sim \mathcal{N}(0, 1) \quad \text{with length } T\end{align}

**2. Bandpass filter:** Apply a 4th-order Butterworth bandpass filter:

\begin{align}e(t) = \text{filtfilt}(b, a, w(t))\end{align}

where $b, a$ are filter coefficients within the passband 
$(f_{\min}, f_{\max})$.

**3. Windowing:** Apply a Hanning window $h(t) \in \mathbb{R}^{T}$ to 
restrict temporal extent:

\begin{align}\tilde{e}(t) = e(t) \cdot h(t)\end{align}

**4. Normalization:** Standardize by peak amplitude:

\begin{align}\hat{e}(t) = \frac{\tilde{e}(t)}{\max(|\tilde{e}(t)|)}\end{align}

**5. Amplitude scaling:** Define signal strength using scalar 
$A \in \mathbb{R}_{+}$:

\begin{align}x(t) = A \cdot \hat{e}(t)\end{align}

Each time series $x(t)$ becomes a row in the source matrix 
$\mathbf{X} \in \mathbb{R}^{N \times T}$, containing all source time courses.



### Multi-Trial Simulation

The simulation supports generating multiple trials by iterating $n_{\text{trials}}$ times.
Within each iteration, the script generates source activity $\mathbf{X}$ with a unique random seed, ensuring reproducible yet varied simulations across trials.

The resulting data is organized as a 3D array: trials $\times$ sources 
$\times$ time points.

<div class="alert alert-info"><h4>Note</h4><p>Although multiple trials can be generated, this tutorial focuses on 
   single-trial analysis since the inverse solvers and uncertainty estimation 
   methods in CaliBrain typically operate on individual trials.</p></div>



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import logging
from pathlib import Path

## The SourceSimulator Class

The :class:`~calibrain.source_simulation.SourceSimulator` class is located in the 
``calibrain/source_simulation.py`` module and serves as the core component 
for generating synthetic brain activity data at the source level.



### Class Methods

The :class:`~calibrain.source_simulation.SourceSimulator` class consists of three core methods:
   - :meth:`~calibrain.source_simulation.SourceSimulator._simulate_erp_waveform`: Internal method for generating individual ERP-like signals
   - :meth:`~calibrain.source_simulation.SourceSimulator._simulate_source_time_courses`: Internal method for creating time courses for multiple sources
   - :meth:`~calibrain.source_simulation.SourceSimulator.simulate`: Main public method for multi-trial simulation. This is the main method that should be used as it wraps the core functionality. Later one can slice specific trials when needed.

The :class:`~calibrain.source_simulation.SourceSimulator` class is part of the CaliBrain package and can be imported directly as follows:




In [ ]:
from calibrain import SourceSimulator

### Basic Usage Example
The :class:`~calibrain.source_simulation.SourceSimulator` class expects a configuration dictionary for ERP parameters and a logger. However if we do not provide these, default values will be used.
The :meth:`~calibrain.source_simulation.SourceSimulator.simulate` function of the :class:`~calibrain.source_simulation.SourceSimulator` class is the main entry point for running the simulation and also expects configuration parameters. If no parameters are provided, default values will be used.

Quick demonstration with default parameters.



In [ ]:
demo_simulator = SourceSimulator()   # use default ERP parameters
x_demo, active_indices_demo = demo_simulator.simulate() # use default simulation parameters

# We can always use the CaliBrain's utility function :func:`~calibrain.utils.inspect_object` to explore any 
# class structure and understand its available attributes and methods:

from calibrain.utils import inspect_object

class_info = inspect_object(demo_simulator, show_private=False)

### Specific Configuration and Simulation

The :class:`~calibrain.source_simulation.SourceSimulator` follows a typical usage pattern:

- **Step 1:** Configure ERP parameters
- **Step 2:** Initialize the simulator
- **Step 3:** Call simulate() method
- **Step 4:** Analyze the returned data



Let's explore a more detailed example with custom configuration.

Configure logging to see simulation progress



In [ ]:
logging.basicConfig(
    level=logging.INFO, # Set to logging.DEBUG for more detailed output
    format="%(asctime)s | %(levelname)s | %(name)s | %(message)s",
    handlers=[logging.StreamHandler()]
)
logger = logging.getLogger("SourceSimulator")

The ERP configuration controls all aspects of the simulated brain activity.




In [ ]:
erp_config = {
    "tmin": -0.5,                # Start time of epoch (seconds before stimulus)
    "tmax": 0.5,                 # End time of epoch (seconds after stimulus)
    "stim_onset": 0.0,           # Stimulus onset time (relative to epoch start)
    "sfreq": 250,                # Sampling frequency in Hz
    "fmin": 1,                   # Minimum frequency for bandpass filter (Hz)
    "fmax": 5,                   # Maximum frequency for bandpass filter (Hz)
    "amplitude": 1.0,            # Amplitude scaling factor
    "random_erp_timing": True,   # Whether to randomize ERP onset and duration
    "erp_min_length": None       # Minimum ERP duration in samples (None = auto)
}

Create and Configure SourceSimulator




In [ ]:
source_simulator = SourceSimulator(
    ERP_config=erp_config,
    logger=logger
)
print(f"- Default units: {source_simulator.source_units}") # -> Am

<div class="alert alert-info"><h4>Note</h4><p>Default units are set to Amperes (Am) which corresponds to FIFF.FIFF_UNIT_AM
   in MNE notation. This ensures compatibility with standard neuroimaging formats.
   See https://github.com/mne-tools/mne-python/blob/main/mne/_fiff/constants.py for details.</p></div>



Now let's generate realistic source activity with multiple trials:

Simulation parameters



In [ ]:
simulation_params = {
    "orientation_type": "fixed",    # Source orientation type
    "n_sources": 50,               # Total number of source locations
    "nnz": 6,                      # Number of active sources per trial
    "n_trials": 3,                 # Number of trials to simulate
    "global_seed": 42              # Seed for reproducibility
}

# Execute the simulation
x_trials, x_active_indices_trials = source_simulator.simulate(**simulation_params)

.. Hint::
   If you are not familiar with ``**kwargs``, you can define the variables explicitly:

```python
orientation_type = "fixed"
n_sources = 50
nnz = 6
n_trials = 3
seed = 42

x_trials, x_active_indices_trials = source_simulator.simulate(
    orientation_type=orientation_type,
    n_sources=n_sources,
    nnz=nnz,
    n_trials=n_trials,
    global_seed=seed
)
```
   See [**kwargs documentation](https://book.pythontips.com/en/latest/args_and_kwargs.html) for more details on this syntax.



Let's inspect the simulation results



In [ ]:
print(f"\nSimulation Results:")
print(f"  - Output shape: {x_trials.shape}")
print(f"    - {x_trials.shape[0]} trials")
print(f"    - {x_trials.shape[1]} sources") 
print(f"    - {x_trials.shape[2]} time points")
print(f"  - Active indices shape: {x_active_indices_trials.shape}")
print(f"  - Data range: [{x_trials.min():.2e}, {x_trials.max():.2e}] Am")

print(f"\nActive Sources by Trial:")
for i, indices in enumerate(x_active_indices_trials):
    print(f"  Trial {i+1}: sources {[int(idx) for idx in sorted(indices)]}")

# Check for non-zero activity only in active sources
trial_0 = x_trials[0]  # First trial
active_sources_0 = x_active_indices_trials[0]
n_nonzero = np.count_nonzero(np.any(trial_0, axis=1))
print(f"\nValidation:")
print(f"  - Sources with non-zero activity: {n_nonzero}")
print(f"  - Expected active sources: {len(active_sources_0)}")
print(f"  - Match: {n_nonzero == len(active_sources_0)}")

## Visualization with CaliBrain

The CaliBrain :class:`~calibrain.visualization.Visualizer` provides sophisticated plotting capabilities for
source data analysis



In [ ]:
from calibrain import Visualizer

# Create visualizer instance with output directory
save_path = Path("tutorial_results")
save_path.mkdir(exist_ok=True)

viz = Visualizer(base_save_path=str(save_path), logger=logger)

<div class="alert alert-info"><h4>Note</h4><p>For an in-depth guide to the :class:`~calibrain.visualization.Visualizer` class and advanced visualization techniques, refer to the `tut-visualization` tutorial.</p></div>



**Plot All Trials**

First, let's visualize source activity across all trials:



In [ ]:
viz.plot_source_signals(
    ERP_config=source_simulator.ERP_config,
    x_trials=x_trials,
    x_active_indices=x_active_indices_trials,
    units=source_simulator.source_units,
    trial_idx=None,  # If None, all trials are plotted
    title="Source Activity - All Trials",
    save_dir="source_simulation",
    file_name="all_trials_overview",
    show=True
)

**Plot Single Trial**

Now let's examine a single trial in detail:



In [ ]:
trial_idx = 0
viz.plot_source_signals(
    ERP_config=source_simulator.ERP_config,
    x_trials=x_trials,
    x_active_indices=x_active_indices_trials,
    units=source_simulator.source_units,
    trial_idx=trial_idx,
    title=f"Source Activity - Trial {trial_idx+1}",
    save_dir="source_simulation", 
    file_name=f"trial_{trial_idx+1}",
    show=True
)

<div class="alert alert-info"><h4>Note</h4><p>The visualization automatically converts units from Amperes (Am) to 
   nanoAmperes (nAm) for better readability. No manual unit conversion is needed.</p></div>



## Advanced ERP Configuration

The SourceSimulator supports various ERP configurations for different 
experimental scenarios. Let's explore different parameter combinations.



**High-Frequency Fast ERPs**

Configuration for simulating high-frequency, short-duration ERPs typical
of early sensory responses:



In [ ]:
fast_erp_config = {
    "tmin": -0.2,
    "tmax": 0.6,
    "stim_onset": 0.0,
    "sfreq": 500,                # Higher sampling rate for fast dynamics
    "fmin": 8,                   # Higher frequency band (8-30 Hz)
    "fmax": 30,
    "amplitude": 2.0,            # Larger amplitude for prominent responses
    "random_erp_timing": True,   # Random timing for realistic variability
    "erp_min_length": 80         # Minimum duration in samples
}

**Low-Frequency Slow ERPs**  

Configuration for simulating low-frequency, long-duration ERPs typical
of cognitive processing:



In [ ]:
slow_erp_config = {
    "tmin": -1.0,
    "tmax": 2.0,
    "stim_onset": 0.0,
    "sfreq": 200,                # Lower sampling rate sufficient for slow dynamics
    "fmin": 0.5,                 # Lower frequency band (0.5-3 Hz)
    "fmax": 3,
    "amplitude": 0.8,            # Moderate amplitude
    "random_erp_timing": False,  # Fixed timing for consistent late components
    "erp_min_length": 150        # Longer minimum duration
}

**Create Simulators**



In [ ]:
simulator_fast = SourceSimulator(ERP_config=fast_erp_config, logger=logger)
simulator_slow = SourceSimulator(ERP_config=slow_erp_config, logger=logger)

print(f"\nCreated specialized simulators:")
print(f"  - Fast ERP simulator: {fast_erp_config['fmin']}-{fast_erp_config['fmax']} Hz")
print(f"  - Slow ERP simulator: {slow_erp_config['fmin']}-{slow_erp_config['fmax']} Hz")

**Generate Comparative Simulations**




In [ ]:
comparison_params = {
    "orientation_type": "fixed",
    "n_sources": 20,
    "nnz": 3,
    "n_trials": 1,
    "global_seed": 42
}

# Generate fast ERPs
x_fast, x_fast_indices = simulator_fast.simulate(**comparison_params)

# Generate slow ERPs (use different seed for variety)
comparison_params["global_seed"] = 84
x_slow, x_slow_indices = simulator_slow.simulate(**comparison_params)

print(f"\nComparison Results:")
print(f"  Fast ERPs shape: {x_fast.shape}") # (n_trials, n_sources, n_timepoints)
print(f"  Slow ERPs shape: {x_slow.shape}") # (n_trials, n_sources, n_timepoints)
print(f"  Fast ERP active sources: {x_fast_indices[0]}")
print(f"  Slow ERP active sources: {x_slow_indices[0]}")

Now lets create comparative visualizations to highlight the differences:

**Fast ERP visualization**



In [ ]:
viz.plot_source_signals(
    ERP_config=simulator_fast.ERP_config,
    x_trials=x_fast,
    x_active_indices=x_fast_indices,
    units=simulator_fast.source_units,
    title="High-Frequency Fast ERPs (8-30 Hz)",
    save_dir="erp_comparison",
    file_name="fast_erp_simulation",
    show=True
)

**Slow ERP visualization**



In [ ]:
viz.plot_source_signals(
    ERP_config=simulator_slow.ERP_config,
    x_trials=x_slow,
    x_active_indices=x_slow_indices,
    units=simulator_slow.source_units,
    title="Low-Frequency Slow ERPs (0.5-3 Hz)",
    save_dir="erp_comparison",
    file_name="slow_erp_simulation", 
    show=True
)

## Integration with CaliBrain Pipeline

The SourceSimulator is designed to work seamlessly with other CaliBrain
components in a complete neuroimaging simulation pipeline.

**Example Pipeline Code**

Here's a conceptual example of how :class:`~calibrain.source_simulation.SourceSimulator` integrates with
other CaliBrain components:



In [ ]:
pipeline_example = '''
# Complete CaliBrain simulation pipeline example

from calibrain import (
    LeadfieldBuilder, SourceSimulator, SensorSimulator, 
    SourceEstimator, UncertaintyEstimator, MetricEvaluator, Visualizer
)

# 1. Build leadfield matrix
leadfield_builder = LeadfieldBuilder(...)
L = leadfield_builder.get_leadfield(
    subject="fsaverage", 
    orientation_type="fixed"
)

# 2. Simulate source activity  
source_simulator = SourceSimulator(ERP_config=erp_config)
x_trials, x_active_indices = source_simulator.simulate(
    orientation_type="fixed",
    n_sources=L.shape[1],  # Match leadfield dimensions
    nnz=5,
    n_trials=10
)

# 3. Simulate sensor measurements
sensor_simulator = SensorSimulator()
y_clean, y_noisy, noise, noise_var = sensor_simulator.simulate(
    x_trials=x_trials,
    L=L,
    alpha_SNR=0.1,
    n_trials=10
)

# 4. Estimate sources
source_estimator = SourceEstimator(solver="gamma_map")
source_estimator.fit(L, y_noisy[0])
x_hat, x_hat_indices, posterior_cov = source_estimator.predict(y_noisy[0])

# 5. Estimate uncertainty (-> credible intervals)
uncertainty_estimator = UncertaintyEstimator()
ci_lower, ci_upper, _, empirical_coverage = \\
    uncertainty_estimator.get_confidence_intervals_data(
        x=x_trials[0],
        x_hat=x_hat,
        posterior_cov=posterior_cov,
        orientation_type="fixed"
    )

# 6. Evaluate performance
metric_evaluator = MetricEvaluator()
metrics = metric_evaluator.evaluate(
    x_true=x_trials[0],
    x_hat=x_hat,
    active_indices_true=x_active_indices[0],
    active_indices_hat=x_hat_indices
)

# 7. Visualize results
visualizer = Visualizer()
visualizer.plot_source_comparison(x_trials, x_hat, x_active_indices)
'''